# <center> 【Kaggle】Telco Customer Churn 电信用户流失预测案例

&emsp;&emsp;<font face="仿宋">在案例的第二部分中，我们详细介绍了常用特征转化方法，其中有些是模型训练之必须，如自然数编码、独热编码，而有些方法则是以提高数据质量为核心、在大多数时候都是作为模型优化的备选方法，如连续变量分箱、数据标准化等。当然，在此之后，我们首先尝试构建一些可解释性较强的模型来进行用户流失预测，即采用逻辑回归和决策树模型来进行预测，并同时详细介绍了两种模型在实战中的调优技巧，在最终模型训练完成后，我们也重点讨论了关于两种可解释性模型建模结果的解释方法。

&emsp;&emsp;<font face="仿宋">从理论上来说，树模型的判别能力是要强于逻辑回归的，但在上一节最后的建模结果中我们发现两个模型的建模并无显著差别，预测准确率都维持在79%-80%之间，这或许说明很多逻辑回归无法正确判别的样本决策树模型也无法判别，据此我们推测，这是一个“入门容易、精通较难”的数据集。当然，如果我们进一步尝试其他“更强”的集成学习算法，如随机森林、XGB、CatBoost等，在当前数据集上的建模结果和逻辑回归也并无太大差异，因此我们亟需通过特征工程方法进一步提升数据集质量，进而提升最终模型效果。

&emsp;&emsp;<font face="仿宋">当然，哪怕是复杂模型在当前数据集上表现出了更好的效果，采用特征工程方法提升数据质量仍是优化建模结果必不可少的部分，正如时下流行的描述那样，“数据质量决定模型上界，而建模过程只是不断逼近这个上界”，特征工程中的一系列提高数据质量的方法、无论是在工业界实践中还是各大顶级竞赛里，都已然成了最为重要的提升模型效果的手段。

<center><img src="https://tva1.sinaimg.cn/large/008i3skNly1gwllgk4wgqj31hr0u0wh4.jpg" alt="image-20211112170651500" style="zoom:15%;" />

&emsp;&emsp;<font face="仿宋">不过，所谓的通过特征工程方法提高数据质量，看似简单但实际操作起来却并不容易。其难点并不在于其中具体操作方法的理解，至少相比机器学习算法原理，特征工程的很多方法并不复杂，特征工程的最大难点在于配合模型与数据进行方法选择、以及各种方法的工程化部署实现。一方面，特征工程方法众多，需要根据实际情况“因地制宜”，但数据的情况千变万化，很多时候需要同时结合数据探索结论、建模人员自身经验以及对各种备选方法的熟悉程度，才能快速制定行之有效的特征工程策略；另一方面，很多特征工程方法不像机器学习算法有现成的库可以直接调用，很多方法、尤其是一些围绕当前数据集的定制方法，需要自己手动实现，而这个过程就对建模人员本身的代码编写能力及工程部署能力提出了更高的要求。总而言之，特征工程是一个实践高度相关的技术，这也是为何课程会在介绍案例的过程中同步介绍特征工程常用方法的原因。

&emsp;&emsp;<font face="仿宋">当然，从宽泛的角度来看，所有围绕数据集的数据调整工作都可以看成是特征工程的一部分，包括此前介绍的缺失值填补、数据编码、特征变换等，这些方法其实都能一定程度提升数据质量，而本节开始，我们将花费一整节的时间来讨论另一类特征工程方法：特征衍生与特征筛选。而该方法通过创建更多特征来提供更多捕捉数据规律的维度，从而提升模型效果。当然特征衍生也是目前公认的最为有效的、能够显著提升数据集质量方法。

# <center>Part 5.目标编码与特征衍生实战

&emsp;&emsp;本阶开始我们将重点讨论特征工程中的特征衍生与特征筛选方法，并借此进一步提升模型效果。首先需要将此前的操作中涉及到的第三方库进行统一的导入：

In [1]:
# 基础数据科学运算库
import numpy as np
import pandas as pd

# 可视化库
import seaborn as sns
import matplotlib.pyplot as plt

# 时间模块
import time

# sklearn库
# 数据预处理
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer

# 实用函数
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

# 常用评估器
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

# 网格搜索
from sklearn.model_selection import GridSearchCV

# 自定义评估器支持模块
from sklearn.base import BaseEstimator, TransformerMixin

# 自定义模块
from telcoFunc import *

# re模块相关
import inspect, re

其中telcoFunc是自定义的模块，其内保存了此前自定义的函数和类，后续新增的函数和类也将逐步写入其中，telcoFunc.py文件随课件提供，需要将其放置于当前ipy文件同一文件夹内才能正常导入。

&emsp;&emsp;接下来导入数据并执行Part 1中的数据清洗步骤。

In [2]:
# 读取数据
tcc = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# 标注连续/离散字段
# 离散字段
category_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
                'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
                'PaymentMethod']

# 连续字段
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
 
# 标签
target = 'Churn'

# ID列
ID_col = 'customerID'

# 验证是否划分能完全
assert len(category_cols) + len(numeric_cols) + 2 == tcc.shape[1]

# 连续字段转化
tcc['TotalCharges']= tcc['TotalCharges'].apply(lambda x: x if x!= ' ' else np.nan).astype(float)
tcc['MonthlyCharges'] = tcc['MonthlyCharges'].astype(float)

# 缺失值填补
tcc['TotalCharges'] = tcc['TotalCharges'].fillna(0)

# 标签值手动转化 
tcc['Churn'].replace(to_replace='Yes', value=1, inplace=True)
tcc['Churn'].replace(to_replace='No',  value=0, inplace=True)

In [3]:
features = tcc.drop(columns=[ID_col, target]).copy()
labels = tcc['Churn'].copy()

接下来即可直接带入数据进行特征衍生。

- 目标编码

&emsp;&emsp;这里的目标编码，就是指将标签在某特征上的分组统计结果作为特征。尽管这个过程看起来并不复杂，无非就是将标签带入进行分组即可，但实际上有两点需要注意，其一是由于目标编码带入了标签数据，而测试集标签未知，因此目标编码是一定需要划分训练集和测试集的，并且将训练集上的编码结果带入测试集。其二需要注意的是，带入标签信息进行特征衍生是极容易造成模型过拟合的。例如极端情况下一条数据一个编码，则目标编码结果如下：

<center><img src="https://s2.loli.net/2022/02/26/5IqomGYduJTsOLB.png" alt="image-20220226210927293" style="zoom:50%;" />

此时训练集准确率为100%，但测试集的标签分布未必和训练集中标签在tenure字段上的分布保持一致，因此模型会有极大的过拟合风险。因此，为了避免目标编码可能导致的过拟合问题，一方面我们要尽可能控制分组时分组的数量（即计量避免过多特征的交叉组合），其二则是可以借助K折交叉统计方法来执行目标编码，以期尽量避免标签的直接泄露。交叉验证统计过程如下。首先假设原始数据如下：

<center><img src="https://s2.loli.net/2022/02/26/X67neFAgdTwRlmY.png" alt="image-20220226220600606" style="zoom:50%;" />

然后对训练集进行5折等分，分为fold1-5，每一折数据包含两条原始数据，此处我们假设ID=1、2的数据划分在fold1内，ID=3、4的数据划分在fold2内，以此类推。然后我们进行五轮运算：第一轮（1st iter）计算时将fold1数据视作验证集（深绿色），仅对fold2-4中的数据（浅绿色）进行Churn在tenure上的分组均值计算，此时计算结果为tenure取值为1时churn均值为0.75，取值为0时churn均值为0.25，并将0.75、0.25作为测试集的预估结果填入fold1的churn列中；然后进行第二轮（2nd iter）运算，仍然重复第一轮的计算过程，只不过此时验证集变为了fold2中数据，我们利用fold1、fold3-5中的数据进行churn在tenure不同取值下的均值计算，并将计算结果填入fold2中；后面三轮计算也以此类推，最终计算结果如下：

<center><img src="https://s2.loli.net/2022/02/26/SyGZVeNFmoB4jlQ.png" alt="image-20220226220604377" style="zoom:50%;" />

最后，我们提取上图中深蓝色框体内的数据，也就是每一轮验证集上的结果组合出新的一列拼在原始数据集中，也就是下图的Churn_tenure_mean_kfold列，当然上图五轮的测试集最终拼接出来的只有训练集上的结果：

<center><img src="https://s2.loli.net/2022/02/27/tBp49kXNOmelJcg.png" alt="image-20220227115816413" style="zoom:50%;" />

&emsp;&emsp;接下来，围绕训练集上的Churn_tenure_mean_kfold对tenure进行分组求均值，然后将算得按照测试集的tenure取值结果填入测试集的Churn_tenure_mean_kfold列中：

<center><img src="https://s2.loli.net/2022/02/27/lKo98urN4pji2Ad.png" alt="image-20220227120356377" style="zoom:50%;" />

至此我们就完成了借助K折交叉统计进行目标编码的基本过程。不难发现，其实相比原始的分组汇总过程，借助K折交叉统计相当于是先在验证集上进行了一轮间接的统计，然后用这些交叉统计的统计结果作为训练集的分组汇总结果，然后再在训练集的分组汇总结果上进行分组统计，并最终将统计结果填入测试集中。相比不带交叉统计过程的分组统计汇总，上述过程相当于添加了一个间接统计的过程，以此来消除目标编码可能带来的过拟合问题。

> 上述过程和K折交叉验证非常类似，只不过上述过程衍生的是K折统计结果，而非建模结果。

> 此外，对于测试集的填充运算，可以利用训练集衍生特征求均值，也可以直接对标签分组求均值，二者并无差异。

## 六、特征衍生实战

&emsp;&emsp;在完成特征衍生方法函数的封装后，接下来，我们考虑尝试调用这些方法，对案例数据集进行特征衍生，并在实践过程中测试衍生特征的实际建模效果。

#### - 特征衍生的核心矛盾

&emsp;&emsp;尽管我们知道特征衍生有效，但很多时候却不知道哪种方法衍生出来的特征有效。并且，由于特征衍生方法众多，而且理论上这些衍生方法只要稍加组合，就能创造“无限”特征，因此这里就会面对一个非常经典的特征衍生难题，即特征衍生效果和特征衍生效率之间如何权衡的问题，也就是说理论上衍生的特征越多、可能有效的衍生特征也越多、特征衍生的最终效果就越好，但特征衍生（以及后续的特征筛选、模型验证）需要耗费大量的算力，我们实际上并不能无止尽的进行尝试，因此，在进行特征衍生之前，我们需要首先考虑的问题就是效果和效率如何权衡的问题。

> 当然，如果算力没有限制，则可以尽可能在更大的范围内进行特征信息的重组、并且可以朝向无限高阶的范围延展，但一方面我们不可能拥有无限的算力，其二哪怕算力没有限制，对于高阶衍生特征的搜索效率也会伴随着阶数上升而迅速下降（越是高阶衍生的特征、有效信息就越少）。因此，效果和效率的权衡，始终是特征衍生过程无法规避的问题。

&emsp;&emsp;如何做好效果和效率的平衡？这就需有一些“先验”的判断，找准了特征衍生的思路、选择对了特征衍生的方法，就能在尽可能短的时间内找到最能提升模型效果的衍生特征，反之则事倍而功半。当然，但凡涉及到思路的问题，一般都是需要根据数据的实际情况来判断接下来应该采取的策略，而最终特征衍生效果如何、特征衍生过程执行效率又如何，则是非常算法工程人员实际建模经验的一件事了。

&emsp;&emsp;正因如此，在后续的案例讲解过程中，我们也将尽可能的尝试更多的方法，帮助大家快速积累实践经验，同时也会尽可能的总结一些特征衍生的思路和框架，帮助大家以更高的效率、找到更好的特征衍生方法。

In [4]:
# 导入特征衍生模块
import features_creation as fc
from features_creation import *

In [5]:
fc?

Type:        module
String form: <module 'features_creation' from 'D:\\Work\\jupyter\\telco\\正式课程\\features_creation.py'>
File:        d:\work\jupyter\telco\正式课程\features_creation.py
Docstring:  
自动化批量特征衍生模块

总共分为四个板块：
Part 1.相关第三方库
Part 2.基本方法实现函数
Part 3.高阶函数辅助函数
Part 4.高阶函数

使用过程中最常调用高阶函数进行批量自动化特征衍生。
高阶函数能够区分执行训练集和测试集的特征衍生过程，
并且支持测试集特征自动补全、目标编码等额外功能，
具体包括：

交叉组合特征衍生函数：
Cross_Combination

多项式特征衍生函数：
Polynomial_Features

分组统计特征衍生函数：
Group_Statistics

目标编码函数：
Target_Encode

时序字段特征衍生函数：
timeSeries_Creation

NLP特征衍生函数：
NLP_Group_Stat


#### - 特征衍生的一般顺序

&emsp;&emsp;首先，我们可以根据此前讨论的特征衍生方法、以及方法之间的关系，制定一个基本的特征衍生方法应用顺序的基本思路：

- Stage 1.时序特征衍生

&emsp;&emsp;首先，如果数据集中存在时序数据，我们可以优先考虑对时序数据进行特征衍生。时序特征衍生过程并不需要依赖其他任何特征，且衍生出来的特征可以作为备选特征带入到交叉组合或者分组统计的过程中。

- Stage 2.多项式特征衍生

&emsp;&emsp;紧接着，我们就需要来进行多项式特征衍生了。多项式特征衍生往往只适用于连续变量或者取值水平较多的有序变量，并且在实际操作过程中，需要注意衍生特征取值大小的问题，如果衍生特征的绝对值过大，则需要进行数据标准化处理。需要注意的是，这里的数据标准化只需针对演变特征进行处理即可，不需要对原始特征进行处理。而具体衍生几阶，一般来说2-3阶都是可以考虑的，如果连续特征较多并且连续特征包括很多有效特征，则可以考虑更高阶的多项式衍生。

- Stage 3.交叉组合特征衍生

&emsp;&emsp;在完成了时序特征衍生和多项式特征衍生之后，接下来我们看进一步考虑交叉组合特征衍生，并且由于衍生特征的稀疏性会伴随着参与组合的特征数量增加而增加，也就是参与交叉组合的特征越多、衍生特征中有效特征占比越少，因此我们会优先考虑两两交叉组合特征衍生，然后再考虑三三组合、甚至考虑四四组合等。同时需要注意，由于交叉组合本身是一种非常通用且执行效率非常高的特征衍生方法，因此一般情况下，需要针对所有原始离散变量和部分时序衍生字段（分类水平较少的时序衍生字段）进行两两组合，而是否需要进行三三组合，则需要根据两两组合的情况来决定。

- Stage 4.分组统计特征衍生

&emsp;&emsp;由于很多时候分组统计特征衍生需要依据交叉组合的结果进行分组，所以分组统计特征衍生一般会放在交叉组合特征衍生之后。同时，多项式的衍生特征也可以作为被分组统计的特征，因此往往在交叉组合和多项式特征衍生之后，才会进行分组统计。

&emsp;&emsp;分组统计特征衍生既是最重要的特征衍生环节（有可能产出非常多的有效特征），同时也是最为复杂的特征衍生环节。对于分组统计特征衍生来说，无论是通常意义的分组统计还是目标编码，最重要的是要找准关键变量（keyCol）。此处关键变量可以是单独的原始变量、也可以是衍生的时序字段、当然也可以双变量（或者多变量）的交叉组合字段。但选取keyCol的过程并不简单，后续我们会在实践过程中详细讨论。

&emsp;&emsp;此外，分组统计和目标编码是需要分开的，一般来说我们会在分组统计阶段验证合适的keyCol，而在目标编码阶段直接利用已经挑选好的keyCol进行标签的分组统计。

- Stage 5.NLP特征衍生

&emsp;&emsp;NLP特征衍生也可以看成是分组统计特征衍生的一种拓展形式，当然也并不是所有的数据集都适合进行NLP特征衍生，同时，NLP特征衍生也基本上可以看成是独立于其他方法的单独方法，如果出现了适合NLP特征衍生的情况，单独执行NLP方法即可，并不存在和此前方法过多的交叉，关键在于判定当前数据集是否适合进行NLP特征衍生。

&emsp;&emsp;接下来，我们就利用此前的Telco数据集来实践上述流程，由于我们现在并未介绍其他特征筛选技术以及更多的集成学习的工业化建模流程，因此特征的筛选过程仍然暂时考虑使用相关系数进行特征有效性评估，并借助此前定义的新特征建模效果判别函数来进行新特征实际效果。